In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from fuzzywuzzy import fuzz
import re
import pandas as pd

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# تنظيف الأسماء
def clean_name(name):
    # إزالة الأحرف غير الضرورية والأرقام
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\d+', '', name)
    # تحويل الحروف إلى صيغة صغيرة
    name = name.lower()
    return name

# تحميل الأسماء والعملاء من ملف CSV
df = pd.read_excel('/content/drive/MyDrive/names.xlsx')

# تنظيف الأسماء
df['cleaned_name'] = df['Customer_name'].apply(clean_name)

# الحد الأدنى لنسبة التشابه المطلوبة
threshold = 80

# العثور على الأسماء المتشابهة
duplicates = []
for i, name1 in enumerate(df['cleaned_name']):
    for j, name2 in enumerate(df['cleaned_name'][i+1:]):
        ratio = fuzz.token_set_ratio(name1, name2)
        if ratio >= threshold:
            duplicates.append((df['Customer_name'][i], df['Customer_name'][j+i+1], ratio))

# إضافة الأسماء المتشابهة إلى عمود جديد
duplicates_df = pd.DataFrame(duplicates, columns=['name1', 'name2', 'ratio'])
df_merged = pd.merge(df, duplicates_df, how='left', left_on=['Customer_name'], right_on=['name1'])
df_merged['duplicates'] = df_merged['Customer_name'].isin(duplicates_df['name1'])
df_merged['duplicates_count'] = df_merged['Customer_name'].apply(lambda x: len(duplicates_df[duplicates_df['name1'] == x]))

# إضافة رمز العميل إلى عمود جديد
df_merged['customer_code_dup'] = ''
for i, row in df_merged.iterrows():
    customer = df[df['Customer_name'] == row['name2']]
    if not customer.empty:
        df_merged.at[i, 'customer_code_dup'] = customer.iloc[0]['Customer_Code']

# حفظ الملف المعدل
df_merged.to_excel('/content/drive/MyDrive/names_with_duplicates_and_customers.xlsx', index=False)

KeyboardInterrupt: ignored